In [15]:
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt


In [16]:
#Load the dataset
(ds_train,ds_test), ds_info = tfds.load(
    "rock_paper_scissors",
    split=["train","test"],
    shuffle_files=True,
    as_supervised= True,
    with_info= True
)
print(ds_info)

tfds.core.DatasetInfo(
    name='rock_paper_scissors',
    full_name='rock_paper_scissors/3.0.0',
    description="""
    Images of hands playing rock, paper, scissor game.
    """,
    homepage='http://laurencemoroney.com/rock-paper-scissors-dataset',
    data_dir='/root/tensorflow_datasets/rock_paper_scissors/3.0.0',
    file_format=tfrecord,
    download_size=219.53 MiB,
    dataset_size=219.23 MiB,
    features=FeaturesDict({
        'image': Image(shape=(300, 300, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=3),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    nondeterministic_order=False,
    splits={
        'test': <SplitInfo num_examples=372, num_shards=1>,
        'train': <SplitInfo num_examples=2520, num_shards=2>,
    },
    citation="""@ONLINE {rps,
    author = "Laurence Moroney",
    title = "Rock, Paper, Scissors Dataset",
    month = "feb",
    year = "2019",
    url = "http://laurencemoroney.com/rock

In [17]:
#Normalize and batch the data
def normalize_img(image,label):
  image = tf.image.resize(image, (300, 300))
  return tf.cast(image,tf.float32)/255.0

batch_size = 64

ds_train = ds_train.map(normalize_img,num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache().shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(normalize_img,num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.cache().shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [18]:
#Define the transfer learning model
base_model = ResNet50V2(include_top=False,weights="imagenet",input_shape=(300,300,3))

In [19]:
base_model.trainable = False
model = Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(3,"softmax")
])

In [20]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50v2 (Functional)         │ (None, 10, 10, 2048)   │    23,564,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │         6,147 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,570,947 (89.92 MB)

 Trainable params: 6,147 (24.01 KB)

 Non-trainable params: 23,564,800 (89.89 MB)

In [21]:
model.build(input_shape=(None, 300, 300, 3))
model.compile(optimizer="adam",loss=tf.keras.losses.sparse_categorical_crossentropy,metrics=["accuracy"])

In [22]:
his = model.fit(ds_train,validation_data=ds_test,epochs=7)

Epoch 1/7


ValueError: None values not supported.